In [ ]:
import warnings
warnings.simplefilter('ignore')

import pyodbc
import pandas as pd

server = 'LAPTOP-8R8HAJSC\\SQLEXPRESS01'
db = 'United Outdoors Source'

connectionString = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={db};Trusted_Connection=yes;'
connection = pyodbc.connect(connectionString)